In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dados_fraude.tsv", sep="\t")

In [4]:
df.head()

,id,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,fecha,monto,fraude
0,ea767f33-fdc6-4722-b0bc-1b393da86870,4,0.7478,40073.59,17.0,0.000000,5.0,BR,34,Kit 30 Esmaltes Anita*você Escolhe As Cores* N...,cat_1df0a5f,0.490599,991.0,88.0,1,NaN,N,2020-03-13 15:33:48,34.39,0
1,b42ca72a-eb30-418b-ac38-9dcaeb12e81e,4,0.5953,399089.80,50.0,0.923535,13.0,AR,15,Faja Lumbar De Trabajo Reforzada 6 Ballenas No...,cat_eb4cc92,0.677752,2259.0,929.0,1,NaN,Y,2020-03-10 20:05:31,5.04,0
2,ee5632e4-3aaa-4c32-9470-e6ebe53be86e,4,NaN,NaN,50.0,0.000000,18.0,BR,40,Cinto De Segurança Do Palio Strada Palio Weeke...,cat_cba5a5d,0.082282,1295.0,452.0,1,NaN,Y,2020-03-10 08:49:54,42.27,0
3,90af7ee4-af51-4202-90b2-cbcbdced88fd,4,0.7638,6032.16,50.0,0.389389,0.0,BR,3,Kit 2 Bolsas + 1 Carteira Feminina Mega Promoç...,cat_cc29487,0.992068,981.0,867.0,1,Y,Y,2020-04-20 13:50:24,19.67,0
4,3a1980f1-be20-4150-93d8-2a01bba098ea,4,0.8274,4836.07,3.0,0.586364,6.0,BR,53,"Kit 2 Suporte Parede Cavaco, Guitarra, Baixo, ...",cat_bb85f6b,0.895140,2324.0,833.0,1,NaN,N,2020-03-30 10:03:55,4.38,0


In [7]:
df[df["fraude"] == 1].iloc[0].to_dict()

{'id': 'd3d56eed-8a0c-4bd7-b163-39d17b4aa3ea',
 'a': 4,
 'b': 0.5879,
 'c': 246212.65,
 'd': 27.0,
 'e': 1.627860465116279,
 'f': 0.0,
 'g': 'AR',
 'h': 41,
 'i': 'Cable Micro Hdmi A Hdmi Kolke 1.8mts C/ Doble Filtro Full Hd',
 'j': 'cat_efed068',
 'k': 0.8372673560737631,
 'l': 1641.0,
 'm': 38.0,
 'n': 1,
 'o': 'N',
 'p': 'N',
 'fecha': '2020-04-07 17:28:24',
 'monto': 3.33,
 'fraude': 1}

## Feature Store

In [17]:
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

import pandas as pd
from datetime import datetime

In [11]:
region = "us-east-1"
boto_session = boto3.Session(region_name=region)
role = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001"
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
prefix = "feature-store"
bucket = f"s3://{default_bucket}/{prefix}"
sagemaker_client = boto_session.client("sagemaker", region_name=region)
fs_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
fs_session = Session(boto_session=boto_session, sagemaker_client=sagemaker_client,
                     sagemaker_featurestore_runtime_client=fs_runtime)

In [19]:
fg_name = "fraude-feature-group"
feat_group = FeatureGroup(name=fg_name, sagemaker_session=fs_session)

In [15]:
train =  pd.read_csv("train.csv")
dev = pd.read_csv("dev.csv")

In [16]:
train.head(2)

,a,b,c,d,e,f,g,h,l,m,n,o,p,monto,hora,text_feat,id,fraude
0,4,0.8083,12610.84,20.0,0.186532,122.0,BR,54,3041.0,93.0,Y,NaN,Y,6.68,22,0.015441,69bdeb76-77f3-4ca2-ab73-9cff19e8f493,0
1,4,0.8118,203622.64,37.0,0.000000,237.0,BR,2,4300.0,712.0,Y,NaN,Y,84.90,22,0.067589,92bc9eb9-29db-4f75-9f5c-177267ee14cf,0


In [49]:
train.loc[:, 'split'] = 'train'
dev.loc[:, 'split'] = 'dev'
df = pd.concat([train, dev])
df['EventTime'] = [datetime.now().timestamp()]*len(df)

In [39]:
df.head(2)

,a,b,c,d,e,f,g,h,l,m,n,o,p,monto,hora,text_feat,id,fraude,split,EventTime
0,4,0.8083,12610.84,20.0,0.186532,122.0,BR,54,3041.0,93.0,Y,NaN,Y,6.68,22,0.015441,69bdeb76-77f3-4ca2-ab73-9cff19e8f493,0,train,1639449111
1,4,0.8118,203622.64,37.0,0.000000,237.0,BR,2,4300.0,712.0,Y,NaN,Y,84.90,22,0.067589,92bc9eb9-29db-4f75-9f5c-177267ee14cf,0,train,1639449111


In [51]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('string')

In [52]:
feat_group.load_feature_definitions(df)

[FeatureDefinition(feature_name='a', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='b', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='c', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='d', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='e', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='f', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='g', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='h', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='l', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='m', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='n', feature_type=<FeatureTypeEnum

In [53]:
feat_group.create(s3_uri=bucket, record_identifier_name='id', event_time_feature_name='EventTime',
                  role_arn="arn:aws:iam::681301531115:role/service-role/AmazonSageMaker-ExecutionRole-20211030T221294", 
                  enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:681301531115:feature-group/fraude-feature-group',
 'ResponseMetadata': {'RequestId': 'e51e003e-cbcd-41e0-bc07-6dc0c3e17848',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e51e003e-cbcd-41e0-bc07-6dc0c3e17848',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Tue, 14 Dec 2021 02:41:55 GMT'},
  'RetryAttempts': 0}}

In [54]:
feat_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:681301531115:feature-group/fraude-feature-group',
 'FeatureGroupName': 'fraude-feature-group',
 'RecordIdentifierFeatureName': 'id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'a', 'FeatureType': 'Integral'},
  {'FeatureName': 'b', 'FeatureType': 'Fractional'},
  {'FeatureName': 'c', 'FeatureType': 'Fractional'},
  {'FeatureName': 'd', 'FeatureType': 'Fractional'},
  {'FeatureName': 'e', 'FeatureType': 'Fractional'},
  {'FeatureName': 'f', 'FeatureType': 'Fractional'},
  {'FeatureName': 'g', 'FeatureType': 'String'},
  {'FeatureName': 'h', 'FeatureType': 'Integral'},
  {'FeatureName': 'l', 'FeatureType': 'Fractional'},
  {'FeatureName': 'm', 'FeatureType': 'Fractional'},
  {'FeatureName': 'n', 'FeatureType': 'String'},
  {'FeatureName': 'o', 'FeatureType': 'String'},
  {'FeatureName': 'p', 'FeatureType': 'String'},
  {'FeatureName': 'monto', 'FeatureType': 'Fractional'},
  {'FeatureName': 'hora', 'Featu

In [56]:
feat_group.ingest(df)

KeyboardInterrupt: 

## Deequ

In [1]:
from pyspark.sql import SparkSession
import pydeequ

In [2]:
spark = (SparkSession
        .builder
        .config("spark.jars.packages", pydeequ.deequ_maven_coord)
        .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
        .getOrCreate()
)

Ivy Default Cache set to: /home/marcos/.ivy2/cache
The jars for the packages stored in: /home/marcos/.ivy2/jars
:: loading settings :: url = jar:file:/home/marcos/miniconda3/envs/sagemaker/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-87ce7324-d343-46b3-bcc0-237d82a7855c;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.1.0_spark-2.4-scala-2.11 in central
	found org.scalanlp#breeze_2.11;0.13.2 in central
	found org.scalanlp#breeze-macros_2.11;0.13.2 in central
	found org.scala-lang#scala-reflect;2.11.8 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.11;0.13.0 in central
	found org.spire-ma

2021-12-24 00:42:34 WARN  Utils:66 - Your hostname, jose-b450gt3 resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface enp3s0)
2021-12-24 00:42:34 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2021-12-24 00:42:34 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.csv("dados_fraude.tsv", header=True, inferSchema=True, sep="\t")

In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- a: integer (nullable = true)
 |-- b: double (nullable = true)
 |-- c: double (nullable = true)
 |-- d: double (nullable = true)
 |-- e: double (nullable = true)
 |-- f: double (nullable = true)
 |-- g: string (nullable = true)
 |-- h: integer (nullable = true)
 |-- i: string (nullable = true)
 |-- j: string (nullable = true)
 |-- k: double (nullable = true)
 |-- l: double (nullable = true)
 |-- m: double (nullable = true)
 |-- n: integer (nullable = true)
 |-- o: string (nullable = true)
 |-- p: string (nullable = true)
 |-- fecha: timestamp (nullable = true)
 |-- monto: double (nullable = true)
 |-- fraude: integer (nullable = true)



In [5]:
from pydeequ.profiles import ColumnProfilerRunner

In [6]:
prof = ColumnProfilerRunner(spark)\
    .onData(df)\
    .run()

2021-12-24 00:42:59 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [7]:
for col, profile in prof.profiles.items():
    print(profile)

NumericProfiles for column: e: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 19788,
    "dataType": "Fractional",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",
    "mean": 0.2406794380590618,
    "maximum": 833.3333333333333,
    "minimum": 0.0,
    "sum": 12033.97190295309,
    "stdDev": 4.067621009876801,
    "approxPercentiles": []
}
NumericProfiles for column: n: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 2,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "1",
            45119,
            0.90238
        ],
        [
            "0",
            4881,
            0.09762
        ]
    ],
    "kll": "None",
    "mean": 0.90238,
    "maximum": 1.0,
    "minimum": 0.0,
    "sum": 45119.0,
    "stdDev": 0.29680016105116935,
    "approxPercentiles": []
}
StandardProfiles for column: j: {
    "completeness": 1.0,
    "approx

In [8]:
from pydeequ.suggestions import *
from pydeequ.repository import *

In [9]:
repository = FileSystemMetricsRepository(spark, "./metrics_repo/metrics.json")
result_key = ResultKey(spark, ResultKey.current_milli_time())

In [15]:
sug = ConstraintSuggestionRunner(spark)\
      .onData(df)\
      .addConstraintRule(CategoricalRangeRule())\
      .addConstraintRule(CompleteIfCompleteRule())\
      .addConstraintRule(NonNegativeNumbersRule())\
      .addConstraintRule(RetainTypeRule())\
      .addConstraintRule(RetainCompletenessRule())\
      .run()

In [16]:
from pydeequ.verification import *
from pydeequ.checks import *

In [17]:
checks = Check(
    spark, CheckLevel.Warning, 
    description="Suggested constraints")

checks.addConstraints([eval("checks"+ct["code_for_constraint"]) for ct in sug["constraint_suggestions"]])

Python Callback server started!


In [18]:
anls = VerificationSuite(spark)\
        .onData(df)\
        .addCheck(checks)\
        .useRepository(repository)\
        .saveOrAppendResult(result_key)\
        .run()

In [19]:
VerificationResult.checkResultsAsDataFrame(spark, anls, pandas=True)

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Suggested constraints,Warning,Success,"CompletenessConstraint(Completeness(e,None))",Success,
1,Suggested constraints,Warning,Success,ComplianceConstraint(Compliance(e is non-negat...,Success,
2,Suggested constraints,Warning,Success,ComplianceConstraint(Compliance(n contained in...,Success,
3,Suggested constraints,Warning,Success,"CompletenessConstraint(Completeness(n,None))",Success,
4,Suggested constraints,Warning,Success,ComplianceConstraint(Compliance(n is non-negat...,Success,
5,Suggested constraints,Warning,Success,"CompletenessConstraint(Completeness(j,None))",Success,
6,Suggested constraints,Warning,Success,"CompletenessConstraint(Completeness(f,None))",Success,
7,Suggested constraints,Warning,Success,ComplianceConstraint(Compliance(a contained in...,Success,
8,Suggested constraints,Warning,Success,"CompletenessConstraint(Completeness(a,None))",Success,
9,Suggested constraints,Warning,Success,ComplianceConstraint(Compliance(a is non-negat...,Success,


In [20]:
VerificationResult.successMetricsAsDataFrame(spark, anls, pandas=True)

,entity,instance,name,value
0,Column,d,Completeness,0.99762
1,Column,d is non-negative,Compliance,1.00000
2,Column,"o contained in Y,N",Compliance,1.00000
3,Column,p,Completeness,1.00000
4,Column,fecha,Completeness,1.00000
5,Column,k,Completeness,1.00000
6,Column,l,Completeness,0.99992
7,Column,fraude is non-negative,Compliance,1.00000
8,Column,m is non-negative,Compliance,1.00000
9,Column,j,Completeness,1.00000
